In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#MovieLens Data

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.sparse.linalg import svds

data_path = '/content/drive/MyDrive/AI_P2'
movies_filename = '/content/drive/MyDrive/AI_P2/movie.csv'
ratings_filename = '/content/drive/MyDrive/AI_P2/rating.csv'

df_movies = pd.read_csv(
    os.path.join(data_path, movies_filename),
    usecols=['movieId', 'title'],
    dtype={'movieId': 'int32', 'title': 'str'})

df_ratings = pd.read_csv(
    os.path.join(data_path, ratings_filename),
    usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
df_ratings.shape

(20000263, 3)

In [ ]:
df_ratings.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [ ]:
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


# Singular Value Decomposition

In [ ]:
df_ratings=df_ratings[:2000000] #take the first 2 million ratings
#create MxN matrix
df_movie_features = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0) #NaN values are filled with 0.0

In [ ]:
df_movie_features.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,128671,128686,128715,128736,128832,128842,128898,128900,128902,128968,128991,129030,129034,129036,129068,129233,129235,129303,129350,129354,129428,129530,129659,129707,129786,129788,129822,129857,130052,130069,130073,130075,130219,130462,130490,130496,130512,130642,130644,130768
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
A = df_movie_features.to_numpy()
user_ratings_mean = np.mean(A, axis = 1)
centered = A - user_ratings_mean.reshape(-1, 1)

## **A = USV^T**

In [ ]:
# 
U, singular_values, V_T = svds(centered, k = 50)

In [ ]:
U

array([[ 0.01068878, -0.00921775, -0.01028504, ..., -0.00232665,
        -0.0053511 , -0.00880595],
       [ 0.00369479, -0.00426444, -0.00194045, ..., -0.00081597,
         0.00162059, -0.00313261],
       [-0.00910756,  0.00383238, -0.0003485 , ..., -0.01183656,
         0.00561438, -0.01114692],
       ...,
       [ 0.03344654,  0.00814001, -0.0064223 , ...,  0.01775382,
        -0.03140629, -0.01667692],
       [ 0.00351431,  0.00217941, -0.00426903, ...,  0.00020301,
        -0.00289936, -0.00299958],
       [-0.00355733, -0.00207085, -0.00279444, ...,  0.0006775 ,
        -0.0015722 , -0.000545  ]], dtype=float32)

In [ ]:
V_T

array([[-9.4745994e-02,  3.5969794e-02,  3.8766559e-02, ...,
        -6.9792346e-05,  3.9069750e-04, -3.2128239e-05],
       [ 2.1622902e-01,  1.0363584e-02,  9.1373045e-03, ...,
        -4.7166197e-04, -5.9748051e-04, -6.0774043e-04],
       [ 5.0549783e-02,  1.3126427e-02,  7.9947263e-03, ...,
        -1.2581376e-03, -9.9417928e-04, -9.9782378e-04],
       ...,
       [ 4.4677924e-02,  4.9580220e-02,  1.3000269e-02, ...,
         1.0197614e-03,  1.1681132e-03,  1.0931707e-03],
       [ 3.0714178e-02,  2.7605044e-02,  2.7576867e-02, ...,
         3.6482033e-04,  3.1626856e-04,  4.1824370e-04],
       [-8.1684217e-02, -3.4953836e-02, -1.3232198e-02, ...,
         2.6181366e-03,  2.6274004e-03,  2.6499394e-03]], dtype=float32)

In [ ]:
singular_values = np.diag(singular_values)

In [ ]:
singular_values

array([[ 193.32753,    0.     ,    0.     , ...,    0.     ,    0.     ,
           0.     ],
       [   0.     ,  195.30287,    0.     , ...,    0.     ,    0.     ,
           0.     ],
       [   0.     ,    0.     ,  196.64787, ...,    0.     ,    0.     ,
           0.     ],
       ...,
       [   0.     ,    0.     ,    0.     , ...,  864.34503,    0.     ,
           0.     ],
       [   0.     ,    0.     ,    0.     , ...,    0.     ,  913.46124,
           0.     ],
       [   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,
        2219.4346 ]], dtype=float32)

In [ ]:
# predict movie ratings
predicted_ratings = np.dot(np.dot(U, singular_values), V_T) + user_ratings_mean.reshape(-1, 1)

In [ ]:
preds_df = pd.DataFrame(predicted_ratings, columns = df_movie_features.columns)
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,128671,128686,128715,128736,128832,128842,128898,128900,128902,128968,128991,129030,129034,129036,129068,129233,129235,129303,129350,129354,129428,129530,129659,129707,129786,129788,129822,129857,130052,130069,130073,130075,130219,130462,130490,130496,130512,130642,130644,130768
0,-1.032371,0.816785,0.065364,-0.069287,-0.089832,0.488378,-0.417961,0.045865,-0.139051,-0.341773,-0.513422,0.178584,0.069932,-0.034303,-0.045440,0.260362,0.069649,0.285616,0.471101,-0.199864,-0.034729,0.439174,-0.025721,0.213960,0.094146,0.060701,0.057206,-0.044556,1.233433,-0.004471,0.095023,2.282895,-0.011613,0.456060,0.021102,0.520967,0.004016,-0.014233,0.176871,0.043323,...,-0.007710,-0.007637,-0.001437,-0.001974,-0.001901,-0.002341,-0.007710,-0.010073,-0.009284,-0.003201,-0.010073,-0.002748,-0.001605,-0.001605,-0.009607,-0.000441,-0.004316,-0.003167,0.000509,-0.005256,-0.007290,-0.001605,-0.014006,-0.002858,-0.001877,-0.001741,-0.001728,-0.004781,-0.004179,-0.011247,0.000436,-0.000913,-0.005743,-0.006718,-0.004173,-0.002836,-0.009607,-0.006603,-0.001093,-0.002193
1,0.878409,0.018589,0.350089,0.069685,0.219142,0.379299,0.404331,0.008962,0.102664,-0.336462,0.312136,0.088315,0.029474,0.091949,0.045356,0.030741,0.429900,0.061929,-0.042158,0.062091,-0.037037,0.128742,0.110056,0.276819,0.333762,0.034210,0.057615,-0.010574,0.326948,-0.026604,0.111464,1.481751,0.002342,0.073127,-0.019877,0.147415,-0.003927,0.020103,-0.161435,-0.000244,...,-0.003447,0.000291,-0.001468,-0.000321,0.000662,-0.000419,-0.003447,0.001479,0.001308,0.001028,0.001479,0.000548,-0.000801,-0.000801,-0.004802,0.001637,0.000272,0.001330,-0.000974,-0.003719,-0.003486,-0.000801,0.001901,0.000244,-0.000653,-0.000727,0.000495,-0.001496,0.000349,0.001411,0.001779,-0.000374,-0.001408,0.002510,0.002424,-0.000837,-0.004802,0.001166,-0.000476,0.000287
2,2.004502,0.853211,-0.124892,0.033102,-0.197885,0.725845,-0.070363,-0.043636,-0.004803,0.117872,0.675622,-0.011991,0.023256,0.035115,-0.085639,1.054156,-0.394634,-0.095324,-0.392495,-0.006021,0.612254,0.300113,-0.104179,0.892095,0.300978,-0.106822,0.052140,-0.153645,0.799835,-0.006502,-0.099880,2.957939,0.010231,0.690258,-0.046066,-0.073028,0.025883,0.007296,0.748284,-0.027530,...,0.014029,0.002355,-0.005867,0.000009,-0.003435,-0.000388,0.014029,-0.001359,-0.001174,0.000396,-0.001359,0.000377,0.000973,0.000973,0.019519,0.010235,0.000436,0.002968,-0.000737,0.005804,-0.004255,0.000973,0.003951,0.000342,0.000840,0.000906,0.000638,-0.003460,-0.000256,0.003040,0.003959,-0.004786,-0.001086,0.001811,0.005158,0.001003,0.019519,-0.001381,-0.001391,0.000514
3,-0.730041,0.575875,0.316375,-0.043769,0.175545,0.804844,0.024103,0.053929,0.182834,1.027780,0.215090,0.053206,0.031952,0.014743,0.069392,0.533745,-0.540016,0.039601,0.485803,0.259154,0.650262,0.308804,0.244486,0.137633,-0.052208,-0.045443,-0.014521,-0.067064,0.017107,-0.024689,0.171829,0.444833,-0.000831,0.061157,-0.058791,-0.131904,0.003973,0.000392,0.206134,-0.037649,...,0.001747,0.001691,0.002540,0.001675,0.002821,0.001738,0.001747,0.004465,0.004178,0.002712,0.004465,0.001991,0.001990,0.001990,0.001691,0.001270,0.001783,0.002540,0.001151,0.001958,0.004704,0.001990,0.001713,0.001938,0.001969,0.001979,0.001728,0.002784,0.001792,0.001756,0.001220,0.001608,0.002619,0.001078,0.003182,0.001051,0.001691,0.001607,0.002382,0.001786
4,1.487823,1.324306,1.370971,0.112845,1.282825,0.726826,1.453524,0.217864,0.285134,1.278284,1.579988,0.208238,0.088085,0.475251,0.035512,-0.176552,1.851012,0.099256,0.402608,0.031267,0.674256,0.260642,0.019003,0.071229,0.663683,0.228993,0.160756,0.229393,0.067370,-0.015627,0.448436,0.985232,-0.006345,2.396329,0.040165,1.618646,0.005433,0.037594,0.310421,0.106340,...,-0.002487,-0.002947,0.004797,0.000543,0.002156,0.000243,-0.002487,0.003953,0.003598,0.002265,0.003953,0.001517,-0.001509,-0.001509,-0.003786,0.000288,0.002172,-0.001100,0.000004,0.001105,-0.005823

# Recommendations

In [ ]:
def recommend_movies(preds_df, userId, movies_df, ratings_df, n=5):
    
    # Get Predictions
    user_row = userId - 1 #account for 0 index
    user_predictions = preds_df.iloc[user_row].sort_values(ascending=False) 
    # Merge Movie Titles
    user_data = ratings_df[ratings_df.userId == (userId)]
    watched = (user_data.merge(movies_df, how = 'left', on = 'movieId').
                     sort_values(['rating'], ascending=False))
    # Recommend
    recommendations = (movies_df[~movies_df['movieId'].isin(watched['movieId'])]).merge(pd.DataFrame(user_predictions).reset_index(), how = 'left', on= 'movieId'
                                                                                ).rename(columns = {user_row: 'Predictions'}
                                                                                ).sort_values('Predictions', ascending = False
                                                                                ).iloc[:n, :-1]
                      

    return watched, recommendations

In [ ]:
watched, recommendations = recommend_movies(preds_df, 3000, df_movies, df_ratings, 15)

In [ ]:
# movies the user has watched an rated
watched.head(10)

,userId,movieId,rating,title
82,3000,2115,5.0,Indiana Jones and the Temple of Doom (1984)
87,3000,2193,5.0,Willow (1988)
39,3000,1193,5.0,One Flew Over the Cuckoo's Nest (1975)
41,3000,1198,5.0,Raiders of the Lost Ark (Indiana Jones and the...
42,3000,1201,5.0,"Good, the Bad and the Ugly, The (Buono, il bru..."
48,3000,1235,5.0,Harold and Maude (1971)
49,3000,1246,5.0,Dead Poets Society (1989)
52,3000,1283,5.0,High Noon (1952)
53,3000,1291,5.0,Indiana Jones and the Last Crusade (1989)
54,3000,1293,5.0,Gandhi (1982)


In [ ]:
# top n recommendations based on user ratings
recommendations

,movieId,title
1801,1961,Rain Man (1988)
1041,1097,E.T. the Extra-Terrestrial (1982)
873,919,"Wizard of Oz, The (1939)"
1191,1270,Back to the Future (1985)
338,356,Forrest Gump (1994)
556,587,Ghost (1990)
2608,2797,Big (1988)
138,150,Apollo 13 (1995)
1180,1259,Stand by Me (1986)
2218,2396,Shakespeare in Love (1998)


# Correlation



In [ ]:
#matrix = df_movie_features
df_movies['title'] = df_movies['title'].apply(lambda old_string: old_string.strip())
df = pd.merge(df_movies, df_ratings, on='movieId', how='inner')
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5


In [ ]:
#create sparse MxN matrix
search_matrix = df.pivot_table(
    index='userId',
    columns='title',
    values='rating'
).fillna(0) #NaN values are filled with 0.0

## Check Data

In [ ]:
#creating average mean rating of movie_title
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
#adding number of ratings on movie
ratings['number_of_ratings'] = df.groupby('title')['rating'].count()
ratings = ratings.reset_index()
ratings.head()

,title,rating,number_of_ratings
0,$ (Dollars) (1971),2.500000,1
1,$5 a Day (2008),2.642857,7
2,$9.99 (2008),4.250000,2
3,'71 (2014),3.200000,5
4,'Hellboy': The Seeds of Creation (2004),3.000000,5


In [ ]:
#most rated movie
ratings['title'] = ratings['title'].apply(lambda old_string: old_string.strip())
ratings.sort_values('number_of_ratings', ascending=False).head(10)

,title,rating,number_of_ratings
11893,Pulp Fiction (1994),4.166567,6688
5527,Forrest Gump (1994),4.043264,6599
13301,"Silence of the Lambs, The (1991)",4.188796,6221
13155,"Shawshank Redemption, The (1994)",4.456364,6199
8062,Jurassic Park (1993),3.671210,5943
13880,Star Wars: Episode IV - A New Hope (1977),4.198512,5443
2241,Braveheart (1995),4.038469,5329
14576,Terminator 2: Judgment Day (1991),3.949971,5267
9588,"Matrix, The (1999)",4.186778,5105
12863,Schindler's List (1993),4.306500,4969


##Making Recommendation Similar to "Input"

In [ ]:
#Making Recommendation by input
input='Rain Man (1988)'
input_rating = search_matrix[input]


In [ ]:
#Finding similar movie
similar_to_input = search_matrix.corrwith(input_rating).sort_values(ascending=False)
similar_to_input.head()

title
Rain Man (1988)              1.000000
Good Will Hunting (1997)     0.441145
Dead Poets Society (1989)    0.419234
Truman Show, The (1998)      0.407984
As Good as It Gets (1997)    0.396372
dtype: float64

##Threshold for min num of ratings

In [ ]:
#create dataframe
sim_input = pd.DataFrame(similar_to_input, columns=['Correlation'])
sim_input = pd.merge(sim_input, df_movies, on='title', how='left')
sim_input.dropna(inplace=True)
sim_input.head()

,title,Correlation,movieId
0,Rain Man (1988),1.000000,1961
1,Good Will Hunting (1997),0.441145,1704
2,Dead Poets Society (1989),0.419234,1246
3,"Truman Show, The (1998)",0.407984,1682
4,As Good as It Gets (1997),0.396372,1784


In [ ]:
#adding in ratings
# sim_input = sim_input.join(ratings['number_of_ratings'])
sim_input = pd.merge(sim_input, ratings, on='title', how='left')
sim_input.head()

,title,Correlation,movieId,rating,number_of_ratings
0,Rain Man (1988),1.000000,1961,3.880943,2419
1,Good Will Hunting (1997),0.441145,1704,4.010462,2772
2,Dead Poets Society (1989),0.419234,1246,3.887679,2021
3,"Truman Show, The (1998)",0.407984,1682,3.828620,2369
4,As Good as It Gets (1997),0.396372,1784,3.785780,2173


In [ ]:
sim_input[sim_input['number_of_ratings']>500].sort_values(by='Correlation',ascending=False).head(10)

,title,Correlation,movieId,rating,number_of_ratings
0,Rain Man (1988),1.000000,1961,3.880943,2419
1,Good Will Hunting (1997),0.441145,1704,4.010462,2772
2,Dead Poets Society (1989),0.419234,1246,3.887679,2021
3,"Truman Show, The (1998)",0.407984,1682,3.828620,2369
4,As Good as It Gets (1997),0.396372,1784,3.785780,2173
5,There's Something About Mary (1998),0.395295,1923,3.527620,2462
6,Big (1988),0.388055,2797,3.599693,1956
7,E.T. the Extra-Terrestrial (1982),0.384350,1097,3.763446,3198
8,Ferris Bueller's Day Off (1986),0.383296,2918,3.933827,2297
9,Ghostbusters (a.k.a. Ghost Busters) (1984),0.382924,2716,3.762990,2983
